In [ ]:
# 使用xDeepFM模型对Avazu CTR进行预估
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from deepctr.models import xDeepFM
from deepctr.inputs import SparseFeat,get_feature_names



In [2]:
file_path = './'
fp_train_f = file_path + "avazu_data_10k.csv" #使用小样本进行训练


In [3]:
##==================== xDeepFM 训练 ====================##
data = pd.read_csv(fp_train_f, dtype={'id':str}, index_col=None)
print('data loaded')


data loaded


In [4]:
print(data.head())

                       id  click      hour    C1  banner_pos   site_id  \
0  1.0000094181510943e+18      0  14102100  1005           0  1fbe01fe   
1  1.0000169349117864e+19      0  14102100  1005           0  1fbe01fe   
2   1.000037190421512e+19      0  14102100  1005           0  1fbe01fe   
3   1.000064072448084e+19      0  14102100  1005           0  1fbe01fe   
4  1.0000679056417042e+19      0  14102100  1005           1  fe8cc448   

  site_domain site_category    app_id app_domain  ... device_type  \
0    f3845767      28905ebd  ecad2386   7801e8d9  ...           1   
1    f3845767      28905ebd  ecad2386   7801e8d9  ...           1   
2    f3845767      28905ebd  ecad2386   7801e8d9  ...           1   
3    f3845767      28905ebd  ecad2386   7801e8d9  ...           1   
4    9166c161      0569f928  ecad2386   7801e8d9  ...           1   

  device_conn_type    C14  C15  C16   C17  C18  C19     C20  C21  
0                2  15706  320   50  1722    0   35      -1   79  
1     

In [5]:
#数据加载
sparse_features = ['C1', 'banner_pos', 'site_domain', 'site_id','site_category','app_id','app_category', 'device_type', 'device_conn_type','C14', 'C15','C16']
target = ['click']

In [6]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print(fixlen_feature_columns)
print(feature_names)

[SparseFeat(name='C1', vocabulary_size=6, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'), SparseFeat(name='banner_pos', vocabulary_size=4, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='banner_pos', group_name='default_group'), SparseFeat(name='site_domain', vocabulary_size=317, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_domain', group_name='default_group'), SparseFeat(name='site_id', vocabulary_size=381, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_id', group_name='default_group'), SparseFeat(name='site_category', vocabulary_size=14, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_category', group_name='default_group'), SparseFeat(name='app_id', vocabulary_size=313, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='app_id', group_name='default_group'), SparseFeat(name='app_category', vocabulary_size=14, embedding_dim=4, use_hash=False,

In [7]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [8]:
# 使用xDeepFM进行训练
model = xDeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
# binary_crossentropy就是logloss
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用xDeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

Train on 6400 samples, validate on 1600 samples


C:\Users\34330\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\34330\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


6400/6400 [==============================] - 9s 1ms/sample - loss: 0.5164 - binary_crossentropy: 0.5164 - val_loss: 0.5060 - val_binary_crossentropy: 0.5060
test RMSE 0.38574603043971817


In [9]:
# 输出LogLoss
from sklearn.metrics import log_loss
score = log_loss(test[target].values, pred_ans)
print("LogLoss", score)


LogLoss 0.4797191536948085


In [10]:
tosub=pd.read_csv("./test.csv", dtype={'id':str}, index_col=None)

DeepCTR version 0.7.4 detected. Your version is 0.7.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.7.4


In [11]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    tosub[feature] = lbe.fit_transform(tosub[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, tosub[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print(fixlen_feature_columns)
print(feature_names)

[SparseFeat(name='C1', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'), SparseFeat(name='banner_pos', vocabulary_size=6, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='banner_pos', group_name='default_group'), SparseFeat(name='site_domain', vocabulary_size=3366, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_domain', group_name='default_group'), SparseFeat(name='site_id', vocabulary_size=2825, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_id', group_name='default_group'), SparseFeat(name='site_category', vocabulary_size=22, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_category', group_name='default_group'), SparseFeat(name='app_id', vocabulary_size=3952, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='app_id', group_name='default_group'), SparseFeat(name='app_category', vocabulary_size=28, embedding_dim=4, use_hash=Fal

In [12]:
tosub_model_input = {name:tosub[name].values for name in feature_names}

In [13]:
print(tosub_model_input)

{'C1': array([2, 2, 2, ..., 2, 2, 2], dtype=int64), 'banner_pos': array([0, 0, 0, ..., 0, 0, 0], dtype=int64), 'site_domain': array([3234, 3197, 3197, ..., 1565, 2590, 3197]), 'site_id': array([ 393,  355,  355, ..., 1064, 1491,  264]), 'site_category': array([20,  1,  1, ...,  3,  5, 20]), 'app_id': array([3643, 3643, 3643, ..., 3643, 2260, 3643]), 'app_category': array([0, 0, 0, ..., 0, 3, 0]), 'device_type': array([1, 1, 1, ..., 1, 1, 1], dtype=int64), 'device_conn_type': array([0, 0, 0, ..., 0, 2, 0], dtype=int64), 'C14': array([ 36, 586, 586, ..., 169, 953, 523], dtype=int64), 'C15': array([3, 3, 3, ..., 2, 3, 3], dtype=int64), 'C16': array([2, 2, 2, ..., 4, 2, 2], dtype=int64)}


In [14]:
print(test_model_input)

{'C1': array([1, 2, 2, ..., 2, 2, 2], dtype=int64), 'banner_pos': array([0, 0, 0, ..., 0, 0, 0], dtype=int64), 'site_domain': array([  5, 244, 143, ..., 301, 283, 301]), 'site_id': array([173, 202, 134, ...,  43, 367,  43]), 'site_category': array([5, 5, 4, ..., 2, 2, 2]), 'app_id': array([293, 311, 293, ..., 293, 293, 293]), 'app_category': array([0, 2, 0, ..., 0, 0, 0]), 'device_type': array([0, 1, 1, ..., 1, 1, 1], dtype=int64), 'device_conn_type': array([0, 0, 0, ..., 0, 0, 0], dtype=int64), 'C14': array([ 87, 241, 103, ...,  63, 139,  65], dtype=int64), 'C15': array([2, 2, 1, ..., 2, 2, 2], dtype=int64), 'C16': array([1, 1, 3, ..., 1, 1, 1], dtype=int64)}


In [16]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [17]:
print(pred_ans)

[[0.12137926]
 [0.09203628]
 [0.15118685]
 ...
 [0.10255313]
 [0.12633428]
 [0.10228744]]


In [20]:
# 使用xDeepFM进行训练
model = xDeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
# binary_crossentropy就是logloss
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )

Train on 6400 samples, validate on 1600 samples


C:\Users\34330\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\34330\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


6400/6400 [==============================] - 9s 1ms/sample - loss: 0.5153 - binary_crossentropy: 0.5153 - val_loss: 0.5020 - val_binary_crossentropy: 0.5020


In [21]:
pred_pro = model.predict(tosub_model_input, batch_size=256)

In [22]:
pred_pro

array([[0.14516968],
       [0.14698768],
       [0.14698768],
       ...,
       [0.13697171],
       [0.14693564],
       [0.14869508]], dtype=float32)

In [23]:
y_xDeep=pd.DataFrame(pred_pro,columns=['click'])

In [25]:
y_xDeep=y_xDeep.set_index(tosub['id'])

In [26]:
print(y_xDeep)

                         click
id                            
10000174058809263569  0.145170
10000182526920855428  0.146988
10000554139829213984  0.146988
10001094637809798845  0.142913
10001377041558670745  0.148210
...                        ...
9998166651591969718   0.137005
9998249812366450951   0.148672
99988023653614546     0.136972
9999086574712596585   0.146936
99992636456518364     0.148695

[4577464 rows x 1 columns]


In [27]:
y_xDeep.to_csv('L15_action_kaggle_xDeepFM.csv')